<center> <h1>PRIMERA PREDICIÓN DEL SUEÑO DE UN PACIENTE</h1> </center>
<center> <h4>Noemi González, Roberto Holgado y Carmen Plaza seco</h4> </center>

[URL PHYSIONET](https://physionet.org/content/sleep-accel/1.0.0/)

In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
import statistics as stats

In [2]:
pacientes_sueño=[]
for file in (os.listdir("labels")):
    pacientes_sueño.append(os.path.join("labels",file))
    pacientes_sueño=sorted(pacientes_sueño)
    
pacientes_frec_card=[]
for file in os.listdir("heart_rate"):
    pacientes_frec_card.append(os.path.join("heart_rate",file))

pacientes_pasos=[]
for file in os.listdir("steps"):
    pacientes_pasos.append(os.path.join("steps",file))

pacientes_movimiento=[]
for file in os.listdir("motion"):
    pacientes_movimiento.append(os.path.join("motion",file))

In [3]:
n=3

In [4]:
n_pac=len(pacientes_frec_card) # Número de pacientes

print('Hay un total de',n_pac,'pacientes')

Hay un total de 31 pacientes


In [5]:
#SUEÑO
sueño=pd.read_csv(pacientes_sueño[n], delimiter=' ')
sueño.columns=["Tiempo","Etiqueta"]

#FRECUENCIA CARDIACA
frec_card=pd.read_csv(pacientes_frec_card[n])
frec_card.columns=["Tiempo","Frec card"]

#MOVIMIENTO
mov = pd.read_csv(pacientes_movimiento[11], delimiter=' ')
mov.columns = ["Tiempo","Ac_x","Ac_y","Ac_z"]

In [6]:
n_frec_card=len(frec_card[frec_card["Tiempo"]>0]) # muestras en frecuencia cardiaca
n_sueño=len(sueño) # muestras en etiquetas del sueño

print('El paciente',n, 'tiene', n_frec_card, 'muestras de frecuencia cardiaca recogidas durante la duración de la PSG y',n_sueño,'etiquetas del sueño')

El paciente 3 tiene 5857 muestras de frecuencia cardiaca recogidas durante la duración de la PSG y 989 etiquetas del sueño


La siguiente celda de código se corresponde con la ventana deslizante que recorrerá la señal fisiológica deseada. La función toma como datos de entrada el tamaño de la ventana en segundos y la señal sobre la que se quiere trabajar.

In [7]:
def get_window(w_size,signal):
    time = signal["Tiempo"]
    f_list=[]
    
    for i in range(n_sueño):
        a = w_size*(1+i)
        b = w_size*(2+i)

        idx=[]
        for t in time:
            if t>a and t<b:
                idx.append(True)
            else:
                idx.append(False)
        
        f=signal[idx]
        f_list.append(f)
        
    return f_list

En todos los casos, se seleccionará una w_size de 30 segundos para tener una característica cada 30s y así coincidir la cantidad de los datos adquiridos por medio del Apple Watch con los datos recogidos por la PSG.

Con la señal de frecuencia cardíaca se han decidido sacar 3 vectores de características: mínimo, máximo y media.

FRECUENCIA CARDIACA (TARDA POCO, sólo es 1 columna)

In [ ]:
frec_card

In [11]:
w_size=30
signal=frec_card

f_list= get_window(w_size,signal)

mean_list=[]
min_list=[]
max_list=[]

for f in f_list:
    mean_list.append(round(f['Frec card'].mean(),2))
    min_list.append(round(f['Frec card'].min(),2))
    max_list.append(round(f['Frec card'].max(),2))

print('Frecuencias mínimas:',min_list[:250])
print('\nFrecuencias máximas:',max_list[:250])
print('\nFrecuencias medias:',mean_list[:250])

print('\nLa longitud de las listas de frecuencias es:',len(min_list),',',len(max_list),'y',len(mean_list))

In [9]:
feat1=min_list
feat2=max_list
feat3=mean_list

MOVIMIENTO (TARDA BASTANTE, son 3 columnas)

In [19]:
mov.head()

,Tiempo,Ac_x,Ac_y,Ac_z
0,-11823.663626,0.072296,-0.773834,-0.627976
1,-11823.643360,0.069687,-0.768402,-0.615784
2,-11823.623725,0.064682,-0.771332,-0.606064
3,-11823.604299,0.057754,-0.764954,-0.606247
4,-11823.593529,0.059326,-0.765961,-0.615524


In [16]:
w_size = 30000
signal = mov

f_list = get_window(w_size,signal)

In [17]:
f_list

[Empty DataFrame
 Columns: [Tiempo, Ac_x, Ac_y, Ac_z]
 Index: [], Empty DataFrame
 Columns: [Tiempo, Ac_x, Ac_y, Ac_z]
 Index: [], Empty DataFrame
 Columns: [Tiempo, Ac_x, Ac_y, Ac_z]
 Index: [], Empty DataFrame
 Columns: [Tiempo, Ac_x, Ac_y, Ac_z]
 Index: [], Empty DataFrame
 Columns: [Tiempo, Ac_x, Ac_y, Ac_z]
 Index: [], Empty DataFrame
 Columns: [Tiempo, Ac_x, Ac_y, Ac_z]
 Index: [], Empty DataFrame
 Columns: [Tiempo, Ac_x, Ac_y, Ac_z]
 Index: [], Empty DataFrame
 Columns: [Tiempo, Ac_x, Ac_y, Ac_z]
 Index: [], Empty DataFrame
 Columns: [Tiempo, Ac_x, Ac_y, Ac_z]
 Index: [], Empty DataFrame
 Columns: [Tiempo, Ac_x, Ac_y, Ac_z]
 Index: [], Empty DataFrame
 Columns: [Tiempo, Ac_x, Ac_y, Ac_z]
 Index: [], Empty DataFrame
 Columns: [Tiempo, Ac_x, Ac_y, Ac_z]
 Index: [], Empty DataFrame
 Columns: [Tiempo, Ac_x, Ac_y, Ac_z]
 Index: [], Empty DataFrame
 Columns: [Tiempo, Ac_x, Ac_y, Ac_z]
 Index: [], Empty DataFrame
 Columns: [Tiempo, Ac_x, Ac_y, Ac_z]
 Index: [], Empty DataFrame
 Columns

In [ ]:
raiz_mov_list=[]

for f in f_list:
    raix_mov_list.append(round(f['Frec card'].mean(),2))

## FEATURES

* **feat1: min_list**. La primera característica que se extrae es la lista de frecuencias mínimas.
* **feat2: max_list**. La segunda característica que se extrae es la lista de frecuencias máximas.
* **feat3: mean_list**. La tercera característica que se extrae es la lista de frecuencias medias.

## OUTCOME

* **labels:**. Lista de etiquetas del sueño

# IDENTIFICACION
* **ID:**. Variable que identifica a cada paciente.

In [ ]:
labels=sueño["Etiqueta"]
id_list=[a]*len(labels)

In [ ]:
datos = {'Min_hr': feat1, "Max_hr": feat2,"Mean_hr":feat3,"Etiquetas":labels,"ID_P":id_list}
datos_paciente = pd.DataFrame(data=datos)